### Libraries Import

In [6]:
import glob
import time
from imutils import build_montages
import matplotlib.pyplot as plt
import numpy as np
import cv2
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Lambda
import keras
from keras.models import Model
from keras.layers import Dense
from keras.layers import Input
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image as kimage
from keras.applications.vgg16 import preprocess_input
%matplotlib inline

Using TensorFlow backend.


### Dataset import

In [7]:
consolidated_jpgdir = '/mnt/data/style_transfers/datasets/consolidated/jpg/'
selected_traversals = ['A000', 'A003', 'A058', 'A059', 'A064', 'A155', 'A159', 'A035', 'A037', 'A079', 'A086', 'A150', 'A151', 'A025', 'A026', 'A062', 'A063', 'A064', 'A138']
filenames = []
for traversal in selected_traversals:
    filenames += [img for img in glob.glob(consolidated_jpgdir + "*" + traversal + "*.jpg")]
filenames = sorted(filenames)
traversal_idx = i = 0
images = np.zeros((len(filenames)//32, 32), dtype=object)
start = time.time()
for filename in filenames:
    images[traversal_idx, i] = filename
    i += 1
    if i == 32:
        traversal_idx += 1
        i = 0
print ('[INFO] {} images loaded in {:.1f} seconds.'.format(len(filenames), (time.time()-start)))

[INFO] 608 images loaded in 0.0 seconds.


#### Getting the horizontal offset from ground truth

In [8]:
consolidated_gtdir = '/mnt/data/style_transfers/datasets/consolidated/gt/'
gt = np.zeros((len(filenames)//32, 32), dtype=int)
#consolidated_gt = consolidated_gtdir + "A*_GT.txt"
gt_filenames = []
for traversal in selected_traversals:
    gt_filenames += [gt for gt in glob.glob(consolidated_gtdir + traversal + "_GT.txt")]
gt_filenames = sorted(gt_filenames)
traversal_idx = j = 0
for filename in gt_filenames:
    data = []
    with open(filename, "r") as f:
        data = f.read()
    data = data.split("\n")
    data = list(filter(None, data))
    for j, col in enumerate(data):
        (horizontal, _) = [int(k) for k in col.split(" ")]
        gt[traversal_idx, j] = horizontal 
    traversal_idx += 1
print("[INFO] Ground truth loaded.")

[INFO] Ground truth loaded.


In [9]:
gt_ = [list(filter(lambda x: x < 500 and x > -500, traversal)) for traversal in gt]
gt_mean = np.mean([np.mean(gt__) for gt__ in gt_])
gt_median = np.median([np.median(gt__) for gt__ in gt_])
gt_min = np.min([np.min(gt__) for gt__ in gt_])
gt_max = np.max([np.max(gt__) for gt__ in gt_])

print(gt_mean, gt_median, gt_min, gt_max)

4.072945019358431 -0.5 -113 376


### Sliding the image 

Use 4, 8, 16, 47, 188, 376px slide window

In [10]:
def cv2_imshow(a, title=None, mode=None, i=None, j=None, k=None, **kwargs):
    a = a.clip(0, 255).astype('uint8')
    #a = cv2.rotate(a, cv2.ROTATE_90_CLOCKWISE)
    # cv2 stores colors as BGR; convert to RGB
    if a.ndim == 3:
        if a.shape[2] == 4:
            a = cv2.cvtColor(a, cv2.COLOR_BGRA2RGBA)
        else:
            a = cv2.cvtColor(a, cv2.COLOR_BGR2RGB)
    if mode == 'subplot':
        plt.subplot(i, j, k)
        return plt.imshow(a, **kwargs)
    else:
        plt.figure(figsize=(12,8))
        plt.title(title)
        return plt.imshow(a, **kwargs)
def imshow(a, title=None, mode=None, i=None, j=None, k=None, **kwargs):
    a = a.clip(0, 255).astype('uint8')
    if mode == 'subplot':
        plt.subplot(i, j, k)
        return plt.imshow(a, **kwargs)
    else:
        plt.figure(figsize=(12,8))
        plt.title(title)
        return plt.imshow(a, **kwargs)
def process_image(img):
    x = np.resize(img, (224,224))
    x = np.expand_dims(img, axis=0)
    x = preprocess_input(x)
    return x
def read_image(img_path):
    #img = kimage.load_img(img_path, target_size=(224, 224))
    img = kimage.load_img(img_path)
    x = kimage.img_to_array(img)
    return x
def get_slices(img_path, offset, slides_count, mode='pos'):
    a = np.zeros((slides_count))
    for i in range(0, slides_count):
        if mode == 'pos':
            a[i] = i*offset
        elif mode == 'neg':
            a[i] = -i*offset
    return a
def slide_image(img, offset):
    if offset > 0:
        img[:,:int(i*offset)] = (228, 35, 157)
    else:
        img[:,:int(-i*offset)] = (228, 35, 157)
    return img

## 1. Generating pairs

In [11]:
def make_pairs(images, gt, offset, threshold, slides_count):
    # initialize two empty lists to hold the (image, image) pairs and
    # labels to indicate if a pair is positive or negative
    pairImagePaths = []
    pairOffsets = []
    pairLabels = []
    
    # loop over all images
    for traversal_idx in range(len(images)):
        if traversal_idx == 0:
            continue
        for i in range(32):
            target_gt = gt[traversal_idx, i]
            if abs(target_gt) > 300:
                continue
            base_path = images[0, i]          
            target_path = images[traversal_idx, i]
            if offset >= 0:     
                slides = get_slices(base_path, offset, slides_count)    
                for j, base_slide in enumerate(slides):
                    pairImagePaths.append([base_path, target_path])
                    pairOffsets.append([base_slide, 0])
                    if abs(abs(target_gt) - j*offset) < threshold:
                        pairLabels.append([1])
                    else:
                        pairLabels.append([0])
            else:
                slides = get_slices(target_path, offset, slides_count, mode='neg')  
                for j, target_slide in enumerate(slides):
                    pairImagePaths.append([base_path, target_path])
                    pairOffsets.append([0, target_slide])
                    if abs(abs(target_gt) - j*offset) < threshold:
                        pairLabels.append([1])
                    else:
                        pairLabels.append([0])

    return (np.array(pairImagePaths), np.array(pairOffsets), np.array(pairLabels))

## 2. Building the Siamese NN

### Pre-trained VGG16 'sister' network architecture:

In [44]:
def build_VGG16_model(input_shape):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    model = Model(inputs=base_model.input, outputs=base_model.get_layer('block5_pool').output)
    return model

### Connected componenets and other help functions

In [62]:
def euclidean_distance(vectors):
    (featsA, featsB) = vectors
    # compute the sum of squared distances between the vectors
    sumSquared = K.sum(K.square(featsA - featsB), axis=1, keepdims=True)
    # return the euclidean distance between the vectors
    return K.sqrt(K.maximum(sumSquared, K.epsilon()))

## 3. Training the network

#### Setting the parameters:

In [16]:
offset = 20
threshold = 40
slides_count = 10
batch_size = 250
epochs = 10
IMG_SHAPE = (224,224)

In [17]:
start = time.time()
(imagePathTrain, offsetTrain, labelTrain) = make_pairs(images, gt, offset, threshold, slides_count)
print ('[INFO] {} pairs of image paths with offsets were generated in {:.1f} seconds.'.format(len(imagePathTrain), (time.time()-start)))

[INFO] 5470 pairs of image paths with offsets were generated in 0.1 seconds.


In [28]:
train_sample_size = 10
test_sample_size = 10
train_indices = np.random.choice([i for i in range(0, len(imagePathTrain))], size=train_sample_size)
test_indices = np.random.choice([i for i in range(0, len(imagePathTrain))], size=test_sample_size)
test_indices = test_indices != train_indices

In [39]:
pairTrain = []
for i in range(train_sample_size):
    j = train_indices[i]
    img1 = read_image(imagePathTrain[j, 0])
    img2 = read_image(imagePathTrain[j, 1])
    offset1 = offsetTrain[j, 0]
    offset2 = offsetTrain[j, 1]
    img1 = slide_image(img1, offset1)
    imgg = slide_image(img2, offset2)
    pairTrain.append([img1, img2])
pairTrain = np.array(pairTrain)
labelTrain = labelTrain[train_indices]

### Training the Siamese network

In [66]:
print("[INFO] building siamese network...")
IMG_SHAPE = (752,480,3)
imgA = Input(shape=IMG_SHAPE)
imgB = Input(shape=IMG_SHAPE)
VGGfeatureExtractor = build_VGG16_model(IMG_SHAPE) # building the network only once since we want to share the weights 
featsA = VGGfeatureExtractor(imgA)
featsB = VGGfeatureExtractor(imgB)

[INFO] building siamese network...


In [67]:
# finally, construct the siamese network
distance = Lambda(euclidean_distance)([featsA, featsB])
outputs = Dense(1, activation="sigmoid")(distance)
model = Model(inputs=[imgA, imgB], outputs=outputs)
print(model.summary())

AttributeError: 'tuple' object has no attribute 'layer'

In [65]:
print("[INFO] compiling model...")
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# train the model
print("[INFO] training model...")
history = model.fit([pairTrain[:, 0], pairTrain[:, 1]], labelTrain[:],
    #validation_data=([pairTest[:, 0], pairTest[:, 1]], labelTest[:]),
    batch_size=batch_size,
    verbose=1,
    epochs=epochs)

#num_of_samples = features.shape[0]    
#names = ['Correct offset', 'Wrong offset']
#Y = np_utils.to_categorical(labels, names)
#x,y = shuffle(features, Y, random_state=2)

# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model6.h5")
print("Saved model to disk")

[INFO] building siamese network...


AttributeError: 'tuple' object has no attribute 'layer'

### Extracting features for test dataset

In [ ]:
# serialize the model to disk
print("[INFO] saving siamese model...")
model.save(config.MODEL_PATH)
# plot the training history
print("[INFO] plotting training history...")
utils.plot_training(history, config.PLOT_PATH)

In [ ]:
score = model.evaluate([pairTrain[:, 0], pairTrain[:, 1]], labelTrain[:], verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

## 4. Testing the network

In [ ]:
### Testing
features = dataset_testing()

num_of_samples = features.shape[0]
labels = np.ones((num_of_samples,),dtype='int64')

json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights("model6.h5")
names = ['Change', 'No change']

Y = np_utils.to_categorical(labels, num_classes)
x_test,y_test = shuffle(features,Y, random_state=2)

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])